In [19]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [20]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [21]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [22]:
system_prompt = "You are the best bank assistent and you are given the websit of banks intrust rates\
you need to suggest the bank which has  highest intrust rates for the customers money and added benifits \
ignoring text that might be navigation related. \
Respond in markdown."

In [23]:
print(system_prompt)

You are the best bank assistent and you are given the websit of banks intrust ratesyou need to suggest the bank which has  highest intrust rates for the customers money and added benifits ignoring text that might be navigation related. Respond in markdown.


In [24]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "please provide a banks that has high fixed intrust rates for fixed deposits in this website \
give me the bank names,interest rates and minimum days should hold deposit to get maximum return\
If it includes additional benifits add them to , then summarize these too and reply in markdown .\n\n"
    user_prompt += website.text
    return user_prompt

In [25]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [26]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [27]:
def display_summary (url):
    summary = summarize (url)
    display (Markdown (summary) )
display_summary("https://www.bankrate.com/landing/savings/rates/?mf_ct_campaign=&pid=sem_savings_google_1&sortprods=&prods=&utm_source=google&utm_medium=cpc&utm_term=where%20can%20i%20get%20the%20highest%20interest%20on%20my%20money&utm_cmpid=20455602196&utm_adgid=155731661674&utm_tgtid=kwd-301113760934&utm_mt=b&utm_dvc=c&utm_ntwk=g&utm_devicemdl=&utm_campaign=sem_savings_google_1&utm_bucket=questions&utm_googleclickid=Cj0KCQiAwOe8BhCCARIsAGKeD56LtbZZVB-k16dMDpBJkk-p9h497YGuMPOjJP_HngbACnqtytfeR2saAsneEALw_wcB&gad_source=1&gclid=Cj0KCQiAwOe8BhCCARIsAGKeD56LtbZZVB-k16dMDpBJkk-p9h497YGuMPOjJP_HngbACnqtytfeR2saAsneEALw_wcB")

Based on the information from Bankrate.com regarding the best savings accounts and fixed deposit (CD) rates, here are some of the banks offering the highest interest rates for fixed deposits:

### Best Banks for Fixed Deposits

| Bank Name               | Interest Rate (APY) | Minimum Days to Hold Deposit | Additional Benefits                              |
|------------------------|---------------------|--------------------------|------------------------------------------------|
| **Marcus by Goldman Sachs** | 5.20%               | 12 months                | No fees, online management                      |
| **Capital One**        | 5.05%               | 12 months                | No minimum deposit, early withdrawal options    |
| **Ally Bank**          | 4.95%               | 12 months                | No maintenance fees, user-friendly app         |
| **CIT Bank**           | 4.85%               | 12 months                | High yield savings account linked, no fees     |
| **Synchrony Bank**     | 4.75%               | 12 months                | Offers ATM access with a savings account       |

### Summary
- **Marcus by Goldman Sachs** offers the highest interest rate of **5.20%** for a minimum holding period of **12 months**. It has no fees and allows easy online management.
- **Capital One** follows with **5.05%**, providing flexibility with no minimum deposit and early withdrawal options.
- Both **Ally Bank** and **CIT Bank** offer good rates with attractive features like no maintenance fees and linked high-yield options, making them suitable for customers looking for accessible savings.
- **Synchrony Bank** offers a competitive rate and includes ATM access, adding convenience for account holders.

Make sure to read any applicable terms and conditions, as interest rates and requirements can vary or change.